In [1]:
import xml.etree.ElementTree as ET
import pymysql.cursors

In [ ]:
thesauro = []

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='fitosanitarios',
                             charset='utf8')

try:
    with connection.cursor() as cursor:
        sql = "SELECT * FROM cultivosenfitosanitarios"
        cursor.execute(sql)
        results = cursor.fetchall()
        
        for row in results:
            thesauro.append((row[0],row[1]))
finally:
    connection.close()

In [ ]:
import xml.etree.ElementTree as ET
import codecs

forbidden_tagged_sentences = []

tree = ET.parse('/Users/b3j90/Downloads/MAE_v0.9.6/annotations/generales.xml')
root = tree.getroot()

obligations = root.findall('./TAGS/prohibition')
for obligation in obligations:
    forbidden_tagged_sentences.append((\
            obligation.attrib["idcorpus"],obligation.attrib["text"]))#, cultivo.attrib["newReference"]))
    

cultivosAnotados = []

tree = ET.parse('/Users/b3j90/Downloads/MAE_v0.9.6/annotations/generales.xml')
root = tree.getroot()

cultivos = root.findall('./TAGS/cultivo')
for cultivo in cultivos:
    if cultivo.attrib["newReference"] == "no" and not cultivo.attrib["text"] == 'fruta':
        cultivosAnotados.append((cultivo.attrib["idthesaurus"], \
                                 cultivo.attrib["idcorpus"],cultivo.attrib["text"]))#, cultivo.attrib["newReference"]))

In [ ]:
def divide(sentence, n):
    ngrams = []
    lWords = sentence.replace(',','').replace(')','').replace("'",'').split(' ')
    i = 0
    while i < len(lWords)- (n-1):
        j = 0
        word = ''
        while j < n:
            word = word + lWords[i+j] + ' '
            j = j +1
        i = i+1
        #print word
        ngrams.append(word)
    return ngrams

def compare(s1):
    stopwords = ['de', 'la', 'el', 'las', 'los', 'del']
    for t in thesauro:
        if len(s1) > 2:
            if s1.strip().lower() == t[1].lower() \
            or s1.strip().lower()[:-1] == t[1].lower() \
            or s1.strip().lower()[:-2] == t[1].lower() \
            or s1.strip().lower() == t[1].lower()[:-1] \
            or s1.strip().lower() == t[1].lower()[:-2]: # playing with plurals
            
                return t[0] # Identifier

def evaluate(ner):
    for gold in cultivosAnotados:
        if gold[:-1] == ner[:-1]:
            return True

In [ ]:
# Accuracy: 0.846 in 13.75 minutes
tp = 0
fp = 0
truePositives = []
completed = []

t0 = time()
for cond in forbidden_tagged_sentences:
    grams = divide(cond[1],1)
    if grams is not None:
        for g in grams:
            res = compare(g)
            if res is not None:
                ner = ( str(res),cond[0], g)
                if (evaluate(ner)):
                    if (str(res), cond[0]) not in completed:
                        truePositives.append((g, cond[0]))
                        completed.append((str(res), cond[0]))
                        tp += 1
                else:
                    if not str(res).startswith('3'):
                        fp += 1
            else:
                

t1 = time()
print("Accuracy: %.3f in %.2f minutes" % (tp * 1.0 /(len(cultivosAnotados)), (t1-t0) / 60))
print("Precision: %.3f" % (tp * 1.0 /(tp + fp)))
print("Recall: %.3f" % (tp * 1.0 /(tp + len(cultivosAnotados))))